In [1]:
!pip install neo4j

     |████████████████████████████████| 192 kB 2.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for neo4j: filename=neo4j-5.14.1-py3-none-any.whl size=267560 sha256=81061a0290307a1e7e3db10e9d78871bfe7e3d4c60645883336dc6e8320b3aae
  Stored in directory: /Users/aina/Library/Caches/pip/wheels/42/c9/78/0502da872e8cc63f812e6d68e2eb661e486f4a5b7ef2731da6
Successfully built neo4j


In [2]:
from neo4j import GraphDatabase

In [7]:
uri = "neo4j://localhost:7687"  # Replace with your URI 
user = "neo4j"                 # Replace with your username
password = "19091998"          # Replace with your password

driver = GraphDatabase.driver(uri, auth=(user, password))

In [11]:
class Neo4jConnection:
    
    def __init__(self, uri, user, password):
        self.__uri = uri
        self.__user = user
        self.__password = password
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__password))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def execute_query(self, query):
        if self.__driver is not None:
            with self.__driver.session() as session:
                return [record for record in session.run(query)]


In [ ]:
# label fields in the snomed ct graph

q = """ MATCH (n:ObjectConcept)WHERE (n.FSN ENDS WITH '(finding)' or n.FSN ENDS WITH '(disorder)' )
        SET n:SP
        RETURN n LIMIT 10 """

In [21]:
conn = Neo4jConnection(uri, user, password)

query = """ MATCH (c:SP)-[:HAS_DESCRIPTION]->(d:Description)
            WHERE d.descriptionType = 'Synonym'
            RETURN c.id AS ConceptId, c.FSN AS FSN_concept, d.term AS Synonym
"""

results = conn.execute_query(query)

results[0]['Synonym']

'Poisoning by pharmaceutical excipient, NOS'

In [ ]:
# Run faiss indexing on all the synonyms, find most similar word

In [ ]:
# find node belonging to the synonym

In [18]:
# Find shortest path between concepts 
query = """ MATCH p = shortestPath((start:SP {id: "249473004"})-[*..10]-(end:SP {id: "102608004"}))
            RETURN length(p) as distance """

results = conn.execute_query(query)

for record in results:
    print(record)

/var/folders/fy/m2y3wbmn1lbcwm2rl2s60d_h0000gp/T/ipykernel_75512/1971303142.py:19: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.__driver.session() as session:


<Record distance=3>


In [20]:
query = """ WITH ["249473004","289161009", "223123009", "289163007", "64379006"] AS endNodeIds
            MATCH (start:SP {id: "102608004"}) 
            UNWIND endNodeIds AS endNodeId
            MATCH (end:SP {id: endNodeId})
            CALL {
                WITH start, end
                MATCH p = shortestPath((start)-[*..10]-(end))
                RETURN p
                ORDER BY length(p) ASC LIMIT 1
            }
            RETURN p, length(p) as distance, end.id as endNodeId, start.FSN, end.FSN as NameEndNode """

results = conn.execute_query(query)

results

/var/folders/fy/m2y3wbmn1lbcwm2rl2s60d_h0000gp/T/ipykernel_75512/1971303142.py:19: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.__driver.session() as session:


[<Record p=<Path start=<Node element_id='4:0c414c0f-b769-4d23-ad25-650efdf1893b:99185' labels=frozenset({'Finding', 'SP', 'SPFindings', 'ObjectConcept'}) properties={'FSN': 'Excessive appetite (finding)', 'nodetype': 'concept', 'effectiveTime': '20080731', 'definitionStatusId': '900000000000073002', 'active': '1', 'history': '[{"id": "102608004", "effectiveTime": "20020131", "active": "1", "moduleId": "900000000000207008", "definitionStatusId": "900000000000074008"}]', 'id': '102608004', 'moduleId': '900000000000207008', 'sctid': '102608004'}> end=<Node element_id='4:0c414c0f-b769-4d23-ad25-650efdf1893b:240538' labels=frozenset({'Finding', 'SP', 'SPFindings', 'ObjectConcept'}) properties={'FSN': 'Altered appetite (finding)', 'nodetype': 'concept', 'effectiveTime': '20080731', 'definitionStatusId': '900000000000073002', 'active': '1', 'history': '[{"id": "249473004", "effectiveTime": "20020131", "active": "1", "moduleId": "900000000000207008", "definitionStatusId": "900000000000074008"}

In [ ]:
# extract the shortest path and handle codes with the exact same length 

In [ ]:
# examples of failed matchings in the app 

In [ ]:
conn.close()